# Trabajo Final LP2

## Integrantes:

- Poma Gómez Diego Alonso
- Giancarlo Kuway
- Manuel Mercedez

In [121]:
# Librerias a instalar 

#pip install pretty-html-table
#pip install selenium

# Descargar

In [153]:
# Ingresa producto y correo electrónico
producto= input("Ingrese el producto a comparar: ")
correo= input("Ingrese su dirección de correo electrónico (ej: example@gmail) ")

Ingrese el producto a comparar: leche
Ingrese su dirección de correo electrónico (ej: example@gmail) k


# Metro

## Integrante:
- Poma Gómez Diego Alonso

In [123]:
# PRUEBA FINAL (todos los productos de Metro)

# Se importan librerias
from selenium.webdriver.common.keys import Keys                  # Para cambiar el nombre de un producto inválido ingresado
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import random
from time import sleep
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
s= Service('C:/Users/Diego/Desktop/Trabajo_Final_LP2/chromedriver.exe')   # Se utiliza comando Service() porque arrojaba error
                                                                          # deprecated .... iperkenel.
driver = webdriver.Chrome(service=s,options = options)

# Se asigna la pagina web a scrapear
driver.get("https://www.metro.pe/") 

# Da click para aceptar términos de un aviso emergente
WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'button.swal2-confirm swal2-styled'.replace(' ','.')))).click()

# Selecciona y escribe el producto en el buscador Metro 
WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'input#search-autocomplete-input'))).send_keys(producto)


# Se verifica si el nombre del producto ingresado es válido

while True:
    try:
        # Click en 'Ver todos los resultados'
        WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR,'a.see-all'.replace(' ','.')))).click()
        break
    except:
        print("No se encontraron resultados, nombre de producto inválido")
        producto=input('Vuelva a ingresar el nombre de producto: ')
        # Se sombrea el texto del producto inválido
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#search-autocomplete-input'))).send_keys(Keys.CONTROL + 'a')
        sleep(1)
        # Se escribe el nuevo nombre de producto ingresado
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#search-autocomplete-input'))).send_keys(producto)
        
sleep(1)
# Hace scroll directo hacia el final de página
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")     

# Espera a que carguen todos los productos de la página

# n_product es la cantidad de productos encontrados 
n_product=driver.find_element('xpath','//span[@class="amount"]').text
n_product=n_product.split()
# Si la cantidad de productos es mayor a 18 , esperar a que cargue la página
if int(n_product[0])>18:
    WebDriverWait(driver, 120).until(
        EC.presence_of_all_elements_located((
            By.XPATH,'//div[@class="shelf-content"]/div[@style="display: none;"]')))  

# Se asigna elementos a cada producto
elementos= driver.find_elements(By.XPATH,'//li[@layout="19ccd66b-b568-43cb-a106-b52f9796f5cd"]')

datos1=[]
# Se extraen el nombre(junto con la cantidad) y el precio
for elem in elementos:
    saco=[]
    try:
        precio= elem.find_element("xpath",'.//span[@class="product-prices__value product-prices__value--best-price"]').text
        nombre= elem.find_element("xpath",'.//a[@class="product-item__name"]').text
        print(nombre)
        print(precio)
        saco.append(nombre)
        saco.append(precio)
        datos1.append(saco)
    except:
        precio = 'NO DISPONIBLE'
        producto = 'NO DISPONIBLE'

No se encontraron resultados, nombre de producto inválido
Vuelva a ingresar el nombre de producto: ahosaojdhasdojh
No se encontraron resultados, nombre de producto inválido
Vuelva a ingresar el nombre de producto: sjdaoij
No se encontraron resultados, nombre de producto inválido
Vuelva a ingresar el nombre de producto: queso
Queso Fresco Sin Lactosa Laive x kg
S/36.90 x kg
Queso Fresco Laive Sbelt Laive x kg
S/35.90 x kg
Queso Mozarella Laive Light 250g
S/13.49
Queso Paria en Tajadas Gloria 180g
S/10.50
Queso Gouda Gloria Tajadas 180 g
S/10.50
Queso Paria Cuisine & Co 400g
S/18.50
Queso Crema Delice Natural Pote 455 g
S/20.90
Queso Tipo Mozzarella Oquendo Molde 250 g
S/9.99
Queso Mantecoso Chugur Molde 300 g
S/17.50
Queso Crema Delice Natural Pote 227 g
S/13.20
Queso Andino Arequipa x kg
S/52.00 x kg
Queso Maasdam Slices President 150g
S/10.20
Queso Fresco de Cabra Duman x Kg
S/70.90 x kg
Queso Crema Delice Natural light Pote 227 g
S/13.20
Queso Fresco Floralp Molde 500 gr
S/22.00
Ques

Pack Jamón Sandwich y Queso Edam San Fernando Paquete 890 g
S/34.90
Pack Jamón Pizza + Queso Edam Otto Kunz Paquete 380 g
S/17.00
Papas en Hojuelas Sabor Queso Cheddar Papa Chips Cuisine & Co Tubo 140 g
S/8.50
Papas Pringles Original Lata 124 g + Papas Pringles Queso Lata 124 g
S/19.70
Mix Galletas de Salvado con Romero + Galletas de Linaza y Ajonjolí + Palitos de Yuca con Queso Caja 230 g
S/14.49
Tabla Quesos Metro 450g
S/18.90
Salsa Mayoqueso Walibí 200g
S/7.60
Cream Cheese Plusa Pote 500 g
S/22.90
Boliquesos Horneados Cheetos Bolsa 130 g
S/5.00
Lonjas Sabor Cheddar Violife Bolsa 200 gr
S/19.90
Cremoso Sabor Original Violife Bandeja 200 gr
S/20.50


In [124]:
# LIMPIEZA Y ESTRUCTURACIÓN DE DATOS DESCARGADOS (METRO)

# Se crea la columna Cantidad
# Se importan librerias
import re
import pandas as pd

total1=[]

for i in datos1:
    saco1=[]   # nombre-precio-cantidad
    saco2=[]   # cantidad
    saco3=[]   # Agrupa los datos de cada producto
    saco1.append(i[0])
    for e in i:
        if e==i[0]:
            x= re.findall('\sx\s',e)    # Posee letra _x_ ?
            y= re.findall('[Pp]ack',e) # Posee palabra P-pack en el texto ?
            z= re.findall('^Mix',e)     # Posee letra Mix al comienzo ?
            w=re.findall('\sx.+',e)     # Coincide desde x_ hasta el final de la oración
            o= re.findall('\s[/]\s',e)  # Posee símbolo '/' 
            u= re.findall('\s[+]\s',e)  # Posee símbolo '+'
            q=re.findall('\d.+',e)      # Coincide con cualquier numero_ hasta el final de la oración
            oq=re.findall('\d+',e)      # Coincide con todos los números enteros
            if len(y)>0:
                saco2.append('Pack') #se almacena pack como cantidad
            elif len(z)>0:
                saco2.append('Mix') # se almacena Mix como cantidad

            elif len(x)>0:
                if len(w)>0:
                    saco2.append(w[0])

            elif len(x)==0:
                if len(oq)==0:
                    if len(u)>0:
                        saco2.append('Duo')
                    else:
                        saco2.append('1 unidad')
                elif len(o)>0:
                    saco2.append(q[0])
                elif len(u)==0:
                    v=re.findall('\d+\s+([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)',e)
                    v1=re.findall('\d+([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)',e)
                    # v y v1 sirven para encontrar condiciones en la oracion 'e'
                    if len(oq)>=2:
                        t2=re.findall('%',e)
                        lp=e.split()    # Separa la oración en palabras
                        a=0     # contador
                        for elem in lp:
                            t=re.findall('^\d.+([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)$',elem)
                            t1=re.findall('^([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)$',elem)

                            if len(t)==1:
                                saco2.append(lp[a])              # Si coincide, agregar a saco2
                            elif len(t1)==1:        
                                saco2.append(lp[a-1]+' '+lp[a])  # de lo contrario, juntarlo con el elemento anterior y agregarlo
                            a=a+1                                # a saco2
                    else:
                        saco2.append(q[0])
                        
                    if len(v)==0 or len(v1)==0:
                        saco2.append(q[0])
                    
                elif len(u)>0:
                    saco2.append('Pack')
            else:
                saco2.append(q[0])
            
            if saco2[0]=='Pack' or saco2[0]=='Mix':
                saco3.append('Metro')
                saco3.append(saco1[0])
            else:
                saco1=saco1[0].replace(saco2[0],'')
                saco3.append('Metro')
                saco3.append(saco1)
                
        elif e==i[1]:
            e=e.split()
            for q in e:
                z=re.findall('\d.+',q)
                if len(z)>0:
                    saco3.append(float(z[0]))
            
    saco3.append(saco2[0])
    total1.append(saco3)        

## Longitud de los datos
# len(total1

# Se crea dataframe para estructurar datos con las columnas Nombre-Precio(S/.)-Cantidad

columnas=['Local','Nombre','Precio (S/.)','Cantidad']
Metro=pd.DataFrame(data=total1,columns=columnas)           

driver.close()
Metro

,Local,Nombre,Precio,Cantidad
0,Metro,Queso Fresco Sin Lactosa Laive,36.90,x kg
1,Metro,Queso Fresco Laive Sbelt Laive,35.90,x kg
2,Metro,Queso Mozarella Laive Light,13.49,250g
3,Metro,Queso Paria en Tajadas Gloria,10.50,180g
4,Metro,Queso Gouda Gloria Tajadas,10.50,180 g
...,...,...,...,...
170,Metro,Salsa Mayoqueso Walibí,7.60,200g
171,Metro,Cream Cheese Plusa Pote,22.90,500 g
172,Metro,Boliquesos Horneados Cheetos Bolsa,5.00,130 g
173,Metro,Lonjas Sabor Cheddar Violife Bolsa,19.90,200 gr


# Plaza Vea


# Tottus


In [135]:
# PRUEBA FINAL (todos los productos de Tottus)

# Se importan librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import random
from time import sleep
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
s= Service('C:/Users/Diego/Desktop/Trabajo_Final_LP2/chromedriver.exe')   # Se utiliza comando Service() porque arrojaba error
                                                                          # deprecated .... iperkenel.
driver = webdriver.Chrome(service=s,options = options)

# Se asigna la pagina web a scrapear
driver.get("https://www.tottus.com.pe/") 

# Selecciona y escribe el producto en el buscador Tottus 
try:
    WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#catalystSearchBar'))).click()
    WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#inputSearchBar'))).send_keys(producto)
    #sleep(1)
    WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'button.jsx-2851002401 searchIcon'.replace(' ','.')))).click()
except:
    WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input.jsx-3290144217 desktopSearchBar'.replace(' ','.')))).send_keys(producto)
    WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'button.jsx-3290144217 searchIcon'.replace(' ','.')))).click()


datos3=[]

while True:
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,'//li[@class="product"]')))
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
    sleep(1)

    elementos=driver.find_elements(By.XPATH,'//li[@class="product"]')
    for elem in elementos:
        saco=[]
        try:
            nombre= elem.find_element("xpath",'.//h2[@class="jsx-369422875 name title mini"]').text
            precio= elem.find_element("xpath",'.//span[@class="list price medium cmrPrice "]').text
            cantidad= elem.find_element("xpath",'.//h3[@class="jsx-369422875 extra text descriptive"]').text
            print(nombre)
            print(precio)
            print(cantidad)
            saco.append(nombre)
            saco.append(precio)
            saco.append(cantidad)
            datos3.append(saco)
        except:
            precio = 'NO DISPONIBLE'
            producto = 'NO DISPONIBLE'

    driver.execute_script("window.scrollBy(0, -800);")
    sleep(1)
    try:
        WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((By.XPATH,'//li[@class="next"]'))).click()
        sleep(2)
    except:
        break

SIX PACK MEZCLA LÁCTEA IDEAL AMANECER 6 UNIDADES
s/ 16.3 UN
IDEAL - PAQUETE 6 UN
SIX PACK LECHE EVAPORADA IDEAL 395 GR
s/ 20.9 UN
IDEAL - PAQUETE 6 UN
LECHE FRESCA ENTERA GLORIA 1 LT
s/ 5.7 UN
GLORIA - 1 LT
LECHE EVAPORADA GLORIA 400 GR
s/ 4.4 UN
GLORIA - 400 GR
PACK LECHE SEMIDESCREMADA GLORIA 3 UNIDADES
s/ 16.4 UN
GLORIA - 3 UNIDADES
SIX PACK LECHE LIGHT GLORIA
s/ 22.4 UN
GLORIA - 6 UNIDADES
LECHE EVAPORADA GLORIA LIGHT 400 GR
s/ 4.7 UN
GLORIA - 400 GR
LECHE FRESCA PASTEURIZADA
s/ 6.5 UN
DANLAC - BOTELLA 900 ML
LECHE CHOCOLATADA GLORIA UHT 900 ML
s/ 5.2 UN
GLORIA - 900 ML
SIX PACK LECHE EVAPORADA GLORIA ENTERA 170 ML
s/ 13 UN
GLORIA - 6 UNIDADES
LECHE SIN LACTOSA LAIVE BOLSA 900 ML
s/ 4.5 UN
LAIVE - BOLSA 900 ML
LECHE FRESCA ENTERA LAIVE 900 ML
s/ 4 UN
LAIVE - BOLSA 900 ML
LECHE GLORIA LIGHT UHT 900 ML
s/ 4.6 UN
GLORIA - 900 ML
LECHE EVAPORADA NIÑOS DEFENSA GLORIA 400 GR
s/ 4.5 UN
GLORIA - 400 GR
LECHE SEMIDESCREM UHT LIGHT 3 UNIDADES
s/ 17.2 UN
GLORIA - 3 UNIDADES
LECHE GLORIA ENTER

CHOCOLATE KISSES CON LECHE Y ALMENDRAS 74 GR
s/ 14.9 UN
HERSHEYS - 74 GR
CHOCOLATE LECHE 190 GR
s/ 19.7 UN
MONTBLANC - ENVASE 190 GR
DULCE DE LECHE HELADERO TOTTUS 550 GR
s/ 16.5 UN
TOTTUS - 550 GR
COBERTURA DE LECHE 300 GR
s/ 19.9 UN
DI-PERUGIA - PAQUETE 300 GR
CHOCOLATE CON LECHE 39% CACAO INNATO 80G
s/ 10.5 UN
INNATO - PAQUETE
SIMILAC 3 KID ABBOTT 1.8 KG
s/ 146.7 UN
SIMILAC - 1.8 KG
PEDIASURE SABOR VAINILLA 220 ML
s/ 8.72 UN
PEDIASURE - 220 ML
CAJA DE TABLETAS DE CHOCOLATE DE LECHE ARCOR 696 GR
s/ 52.92 UN
ARCOR - EMPAQUE 58 GR
CHOCOLATE DE LECHE CON CASTAÑAS TOSTADAS
s/ 8.3 UN
BRAVI - CAJA 64 G
FORTIFICADA CHOCOLATE NESQUIK 400 GR
s/ 12.5 UN
NESQUIK - LATA 400 GR
CHOCOLATE DE LECHE LINDT LINDOR 200 GR
s/ 49.9 UN
LINDT - 200 GR
ENFAGROW PROMENTAL 1.35 KG
s/ 109.8 UN
ENFAGROW - 1.35 KG
ANDEAN SUPER BEBIDA NUNATURA CHOCOLATE 200 GR
s/ 12.2 UN
NUNATURA - UNIDAD
SUPER BEBIDA CHOCOLATE NUNATURA 350 GR
s/ 18.1 UN
NUNATURA - DOYPACK 350 GR
SUPLEMENTO CHOCOLATE NESQUIK 200 GR
s/ 8.5 UN
NESQ

SUPLEMENTO TRIPLESURE CHOCOLATE PEDIASURE 1.6 K GR
s/ 154.7 UN
PEDIASURE - 1.6 KG
FOUR PACK ENSURE ADVANCE VAINILLA 220 ML
s/ 34 UN
ENSURE - 220 ML
CARAMELOS OH! AMBROSOLI 100 UNIDADES
s/ 19.7 UN
AMBROSOLI - BOLSA 100 UNDS
SUPLEMENTO GRANULADO MILO 400 GR + MEZCLA LACTEA IDEAL AMANECER 395 GR
s/ 18.3 UN
MILO - LATA 400 GR
SIMILAC 1 X 850G
s/ 123.8 UN
SIMILAC - UNIDAD
FOUR PACK PEDIASURE VAINILLA 220 ML
s/ 31.3 UN
PEDIASURE - ENVASE 220 ML
SIMILAC MAMA X 400G
s/ 53.9 UN
SIMILAC - UNIDAD
CHOCOLATE PARA TAZA CON CANELA Y CLAVO SOL DEL CUSCO 90 GR
s/ 2.2 UN
SOL-DEL-CUSCO - BARRA 90 GR
FORMULA SIMILAC 2 X 850G
s/ 120.3 UN
SIMILAC - UNIDAD
CHOCOLATE PARA TAZA TRADICIONAL SOL DEL CUSCO 90 GR
s/ 2.2 UN
SOL-DEL-CUSCO - BARRA 90 GR
ALIMENTO GRANULADO MILO ACTIV GO 400 GR
s/ 15.9 UN
MILO - LATA 400 GR
ENFAMIL PREMIUM 2 1,1 KG
s/ 179.1 UN
ENFAMIL - CAJA 1.1 KG
TÉ DE COCO TROPICAL NATURES HEART 20 UNIDADES
s/ 11.5 UN
NATURES-HEART - CAJA 20 UNDS
ENFAMIL CONFORT PREMIUM 375 GR
s/ 73.1 UN
ENFAMIL - P

In [132]:

# LIMPIEZA Y ESTRUCTURACIÓN DE DATOS DESCARGADOS (Tottus)

# Se importan librerias
import re
import pandas as pd

total3=[]

for i in datos3:
    saco1=[]   # nombre-precio-cantidad
    saco1.append('Tottus')
    for e in i:
        if e==i[0]:
            saco1.append(e)
        if e==i[1]:
            e=e.split()
            for q in e:
                z=re.findall('\d.+',q)
                z1=re.findall('\d+',q)
                if len(z)>0: 
                    saco1.append(float(z[0]))
                elif len(z1)>0:
                    saco1.append(float(z1[0]))
        if e==i[2]:
            saco1.append(e)

    total3.append(saco1)        

# Se crea dataframe con datos ya estructurados con las columnas Nombre-Precio(S/.)-Cantidad

columnas=['Local','Nombre','Precio (S/.)','Cantidad']
Tottus=pd.DataFrame(data=total3,columns=columnas)           
driver.close()
Tottus

,Local,Nombre,Precio (S/.),Cantidad
0,Tottus,SIX PACK MEZCLA LÁCTEA IDEAL AMANECER 6 UNIDADES,16.3,IDEAL - PAQUETE 6 UN
1,Tottus,SIX PACK LECHE EVAPORADA IDEAL 395 GR,20.9,IDEAL - PAQUETE 6 UN
2,Tottus,LECHE FRESCA ENTERA GLORIA 1 LT,5.7,GLORIA - 1 LT
3,Tottus,LECHE EVAPORADA GLORIA 400 GR,4.4,GLORIA - 400 GR
4,Tottus,PACK LECHE SEMIDESCREMADA GLORIA 3 UNIDADES,16.4,GLORIA - 3 UNIDADES
...,...,...,...,...
330,Tottus,MEZCLA LÁCTEA IDEAL CREMOSITA LATA 6 UNIDADES,20.0,IDEAL - LATAS 6 UNIDADES
331,Tottus,CREMA DE LECHE PASTEURIZADA PIAMONTE 236 ML,6.5,PIAMONTE - ENVASE 236 ML
332,Tottus,BATIDO NUTRICIONAL BENET VAINILLA 400 GR,47.9,BENET - EMPAQUE
333,Tottus,BATIDO NUTRICIONAL BENET VAINILLA 800 GR,89.9,BENET - EMPAQUE


#  Comparación

In [127]:
# Ordena de menor a mayor de
M_orden=Metro.sort_values(by='Precio (S/.)') 
T_orden=Tottus.sort_values(by='Precio (S/.)') 
# Se selecciona el primer y ultimo producto de la tabla anterior
M_altobajo= M_orden.iloc[[0,-1]] 
T_altobajo= T_orden.iloc[[0,-1]]
# Se crea nueva columna que indica la magnitud del precio (más alto y más bajo)
T_altobajo=T_altobajo.assign(Precio_mag =['Precio más Bajo','Precio más Alto'])
M_altobajo=M_altobajo.assign(Precio_mag =['Precio más Bajo','Precio más Alto'])
T_altobajo.index = ['0', '1']
M_altobajo.index = ['0', '1']

# Se unen las tablas (pd.concat) y se muestra de forma horizontal
comparacion1=pd.concat([M_altobajo,T_altobajo],axis=1)      # Comparación en columnas
#comparacion=pd.concat([M_altobajo,T_altobajo],axis=0)     # Comparación en filas

comparacion1

,Local,Nombre,Precio,Cantidad,Precio_mag,Local,Nombre,Precio,Cantidad,Precio_mag
0,Metro,Queso Fundido untable Bonlé jamón sachet,3.0,80 g,Precio más Bajo,Tottus,YOGURT FRESA VASO YOPI,0.9,LAIVE - ENVASE 80 GR,Precio más Bajo
1,Metro,Queso Parmesano Laive,110.0,x kg,Precio más Alto,Tottus,QUESO SUIZO GRUYERE EMMI KG,290.0,EMMI - PESO APROX 200 G,Precio más Alto


In [128]:
# Comparación desde el precio más bajo al precio más alto en todos los productos encontrados en las dos localidades
comparacion2=pd.concat([M_orden,T_orden],axis=0) 
comparacion2.sort_values(by='Precio (S/.)')

,Local,Nombre,Precio,Cantidad
422,Tottus,YOGURT FRESA VASO YOPI,0.90,LAIVE - ENVASE 80 GR
425,Tottus,YOGURT GLORIA BATTISHAKE FRESA MASHA Y EL OSO,1.65,GLORIA - 120 GR
421,Tottus,YOGURT GLORIA LÚCUMA 180 GR,1.80,GLORIA - 180 GR
420,Tottus,YOGURT GLORIA DURAZNO 180 GR,1.80,GLORIA - 180 GR
423,Tottus,YOGURT GLORIA VAINILLA 180 GR,1.80,GLORIA - 180 GR
...,...,...,...,...
310,Tottus,QUESO GORGONZOLA PICCANTE VENTA AL PESO,139.90,PERLA - CUÑA 150 GR
52,Tottus,QUESO MADURADO ALTARTUFO MARENCHINO KG,140.00,PERLA - PESO APROX 200 G
53,Tottus,QUESO GORGONZOLA PICANTE IGOR KG,149.90,IGOR - PESO APROX 150 GR
28,Tottus,QUESO SUIZO EMMENTAL EMMI KG,250.00,EMMI - PESO APROX 200 G


In [129]:
# Envío de de los resultados al correo ingresado anteriormente

from pretty_html_table import build_table
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import getpass

# Se establece un usuario y contraseña por defecto quien va a enviar el correo
# usuario: jossemen01@gmail.com
# contraseña: mxszphkwuhexmfue       <- contraseña generado para aplicaciones autorizadas en la cuenta gmail quien envía.

sender_address = 'jossemen01@gmail.com'
sender_pass = 'mxszphkwuhexmfue'
receiver_address = correo

# El cuerpo del mensaje puede ser html, json, texto puro
html1 = build_table(comparacion1, 'blue_light')
html2 = build_table(comparacion2, 'blue_light')

body= 'Aquí se presentan los resultados con el precio más alto y más bajo' + html1 +'Aquí se presentan los resultados de los precios entre las tres localidades' + html2

message = MIMEMultipart("alternative")  # Tipos de mensaje multiparte como html
message['From'] = sender_address
message['To'] = receiver_address
message['Subject'] = 'Resultados de la comparación de producto en 3 localidades'   

message.attach(MIMEText(body, 'html'))   # plain -> texto puro(plano)   html-> formato html  json...
# Establece el tipo de correo y su puerto (587)
session = smtplib.SMTP('smtp.gmail.com', 587)
# Inicia sesión
session.starttls()
session.login(sender_address, sender_pass)
text = message.as_string()
# Envío de correo
session.sendmail(sender_address, receiver_address, text)
session.quit()

print('Los resultados de la comparación fueron enviados con éxito a su correo')

Los resultados de la comparación fueron enviados con éxito a su correo
